In [ ]:
%run main.py
%load_ext autoreload
%autoreload 2

!mkdir -p {DATA_DIR} {NAVEC_DIR} {MODEL_DIR}
s3 = S3()

In [ ]:
if not exists(NERUS):
    s3.download(S3_NERUS, NERUS)
    
if not exists(NAVEC):
    !wget {NAVEC_URL} -O {NAVEC}
    s3.download(S3_TAGS_VOCAB, TAGS_VOCAB)

In [ ]:
navec = Navec.load(NAVEC)

In [ ]:
words_vocab = Vocab(navec.vocab.words)
shapes_vocab = Vocab([PAD] + SHAPES)

In [ ]:
# lines = load_gz_lines(NERUS)
# lines = log_progress(lines, total=NERUS_TOTAL)
# items = parse_jl(lines)
# markups = (MorphMarkup.from_json(_) for _ in items)

# tags = set()
# for markup in markups:
#     for token in markup.tokens:
#         tags.add(token.tag)

# tags = [PAD] + sorted(tags)
# tags_vocab = Vocab(tags)
# tags_vocab.dump(TAGS_VOCAB)

tags_vocab = Vocab.load(TAGS_VOCAB)

In [ ]:
torch.manual_seed(SEED)
seed(SEED)

In [ ]:
word = NavecEmbedding(navec)
shape = Embedding(
    vocab_size=len(shapes_vocab),
    dim=SHAPE_DIM,
    pad_id=shapes_vocab.pad_id
)
emb = TagEmbedding(word, shape)
encoder = TagEncoder(
    input_dim=emb.dim,
    layer_dims=LAYER_DIMS,
    kernel_size=KERNEL_SIZE,
)
morph = MorphHead(encoder.dim, len(tags_vocab))
model = Morph(emb, encoder, morph)

model = model.to(DEVICE)

criterion = flatten_cross_entropy

In [ ]:
lines = load_gz_lines(NERUS)
lines = log_progress(lines, total=NERUS_TOTAL)
items = parse_jl(lines)
markups = (MorphMarkup.from_json(_) for _ in items)

encode = TagTrainEncoder(
    words_vocab, shapes_vocab, tags_vocab,
    seq_len=256,
    batch_size=64,
    shuffle_size=1000,
)
batches = encode(markups)
batches = [_.to(DEVICE) for _ in batches]

size = 10
batches = {
    TEST: batches[:size],
    TRAIN: batches[size:]
}

In [ ]:
board = MultiBoard([
    TensorBoard(BOARD_NAME, RUNS_DIR),
    LogBoard()
])
boards = {
    TRAIN: board.section(TRAIN_BOARD),
    TEST: board.section(TEST_BOARD),
}

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, LR_GAMMA)

In [ ]:
meters = {
    TRAIN: MorphScoreMeter(),
    TEST: MorphScoreMeter(),
}

for epoch in log_progress(range(EPOCHS)):
    model.train()
    for batch in log_progress(batches[TRAIN], leave=False):
        optimizer.zero_grad()
        batch = process_batch(model, criterion, batch)
        batch.loss.backward()
        optimizer.step()
    
        score = score_morph_batch(batch)
        meters[TRAIN].add(score)

    meters[TRAIN].write(boards[TRAIN])
    meters[TRAIN].reset()

    model.eval()
    with torch.no_grad():
        for batch in log_progress(batches[TEST], leave=False, desc=TEST):
            batch = process_batch(model, criterion, batch)
            score = score_morph_batch(batch)
            meters[TEST].add(score)

        meters[TEST].write(boards[TEST])
        meters[TEST].reset()
    
    scheduler.step()
    board.step()

In [ ]:
# [2020-04-11 07:43:31]    0 0.1641 01_train/01_loss
# [2020-04-11 07:43:31]    0 0.9529 01_train/02_acc
# [2020-04-11 07:43:32]    0 0.1220 02_test/01_loss
# [2020-04-11 07:43:32]    0 0.9626 02_test/02_acc
# [2020-04-11 07:45:27]    1 0.1163 01_train/01_loss
# [2020-04-11 07:45:27]    1 0.9643 01_train/02_acc
# [2020-04-11 07:45:27]    1 0.1118 02_test/01_loss
# [2020-04-11 07:45:27]    1 0.9657 02_test/02_acc
# [2020-04-11 07:47:23]    2 0.1094 01_train/01_loss
# [2020-04-11 07:47:23]    2 0.9663 01_train/02_acc
# [2020-04-11 07:47:23]    2 0.1073 02_test/01_loss
# [2020-04-11 07:47:23]    2 0.9670 02_test/02_acc
# [2020-04-11 07:49:19]    3 0.1057 01_train/01_loss
# [2020-04-11 07:49:19]    3 0.9673 01_train/02_acc
# [2020-04-11 07:49:19]    3 0.1045 02_test/01_loss
# [2020-04-11 07:49:19]    3 0.9680 02_test/02_acc
# [2020-04-11 07:51:14]    4 0.1032 01_train/01_loss
# [2020-04-11 07:51:14]    4 0.9681 01_train/02_acc

# [2020-04-11 07:51:14]    4 0.1027 02_test/01_loss
# [2020-04-11 07:51:14]    4 0.9683 02_test/02_acc

In [ ]:
# tags_vocab.dump(TAGS_VOCAB)
# model.emb.shape.dump(MODEL_SHAPE)
# model.encoder.dump(MODEL_ENCODER)
# model.morph.dump(MODEL_MORPH)

# s3.upload(TAGS_VOCAB, S3_TAGS_VOCAB)
# s3.upload(MODEL_SHAPE, S3_MODEL_SHAPE)
# s3.upload(MODEL_ENCODER, S3_MODEL_ENCODER)
# s3.upload(MODEL_MORPH, S3_MODEL_MORPH)